<a href="https://colab.research.google.com/github/MBrandao07/Spark_SQL/blob/main/Manipula%C3%A7%C3%A3o_Dados_SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Introdução**

O objetivo deste projeto é mostrar como o Spark e o Spark SQL facilitam o trabalho com dados.

Vou apresentar como essas ferramentas ajudam a manipular grandes volumes de informações e a criar novas variáveis de forma rápida e prática.

A ideia é destacar como o Spark combina a potência de processamento com a facilidade do SQL, tornando o processo de análise muito mais eficiente.

In [1]:
# instalando o pacote do pyspark
!pip install pyspark

In [2]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# criando nossa a sessão com o SparkSession
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Manipulação de dados no Pyspark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [5]:
# importando a base de dados
df = spark.read.parquet('/content/dados_veiculos_sample')

In [9]:
df.show(10, truncate=False)

+------+---------------------------------+-----------+----+-----------------+-----------------+---+-------------------+-------------------+-------+-----------------+
|marca |modelo                           |engine_size|ano |valor_tabela_fipe|flag_veiculo_novo|uf |data_cadastro      |cor                |placa  |__index_level_0__|
+------+---------------------------------+-----------+----+-----------------+-----------------+---+-------------------+-------------------+-------+-----------------+
|Acura |NSX 3.0                          |3.0        |1995|43779.0          |False            |MT |2005-09-02 14:40:54|Orquídea média     |4RR2KR0|0                |
|Acura |NSX 3.0                          |3.0        |1994|42244.0          |False            |MT |1998-11-01 11:58:46|Azul aço           |5C6NIZ1|1                |
|Acura |NSX 3.0                          |3.0        |1992|39028.0          |False            |AP |2014-07-03 20:32:21|Ametista           |3PRJCPK|3                |
|Acu

In [10]:
# verificando o schema do df
df.printSchema()

root
 |-- marca: string (nullable = true)
 |-- modelo: string (nullable = true)
 |-- engine_size: double (nullable = true)
 |-- ano: long (nullable = true)
 |-- valor_tabela_fipe: double (nullable = true)
 |-- flag_veiculo_novo: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- data_cadastro: timestamp_ntz (nullable = true)
 |-- cor: string (nullable = true)
 |-- placa: string (nullable = true)
 |-- __index_level_0__: long (nullable = true)



In [31]:
# apagando a coluna "__index_level_0__"
df = df.drop('__index_level_0__')

In [32]:
# criando um data frame com os dados de algumas colunas
df_marca = df.select('marca', 'modelo', 'ano', 'valor_tabela_fipe')

In [33]:
df_marca.show(10, truncate=False)

+------+---------------------------------+----+-----------------+
|marca |modelo                           |ano |valor_tabela_fipe|
+------+---------------------------------+----+-----------------+
|Acura |NSX 3.0                          |1995|43779.0          |
|Acura |NSX 3.0                          |1994|42244.0          |
|Acura |NSX 3.0                          |1992|39028.0          |
|Acura |Legend 3.2/3.5                   |1998|27132.0          |
|Acura |Legend 3.2/3.5                   |1995|20393.0          |
|Acura |Legend 3.2/3.5                   |1993|15156.0          |
|Acura |Legend 3.2/3.5                   |1992|14440.0          |
|Acura |Integra GS 1.8                   |1991|10785.0          |
|Agrale|MARRUÁ AM 50 2.8 140cv TDI Diesel|2010|64744.0          |
|Agrale|MARRUÁ AM 50 2.8 140cv TDI Diesel|2008|55772.0          |
+------+---------------------------------+----+-----------------+
only showing top 10 rows



In [34]:
# importando as libs para manipulação das colunas
from pyspark.sql.functions import col, lit, pow, round, asc, desc

#### Considerando que a coluna "valor_tabela_fipe" é o valor do carro no ano em que foi lançado e que o carro perde 3% do valor por ano, então vamos calcular qual a porcentagem de perde e o valor final considerando o ano atual (2025)

In [35]:
# criando a coluna de anos de uso
ano_atual = 2025
df_marca = df_marca.withColumn("anos_uso", lit(ano_atual) - col("ano"))

# calculando o percentual de depreciação acumulada
df_marca = df_marca.withColumn("percentual_depreciacao", 1 - pow(1 - 0.03, col("anos_uso")))

# aplicando a depreciação no valor original
df_marca = df_marca.withColumn("valor_2025", col("valor_tabela_fipe") * (1 - col("percentual_depreciacao")))

# (Opcional) Arredondar o valor final
from pyspark.sql.functions import round
df_marca = df_marca.withColumn("valor_2025", round(col("valor_2025"), 2))
df_marca = df_marca.withColumn("percentual_depreciacao", round(col("percentual_depreciacao") * 100, 2))

In [36]:
df_marca.show(10)

+------+--------------------+----+-----------------+--------+----------------------+----------+
| marca|              modelo| ano|valor_tabela_fipe|anos_uso|percentual_depreciacao|valor_2025|
+------+--------------------+----+-----------------+--------+----------------------+----------+
| Acura|             NSX 3.0|1995|          43779.0|      30|                  59.9|  17555.69|
| Acura|             NSX 3.0|1994|          42244.0|      31|                  61.1|  16431.94|
| Acura|             NSX 3.0|1992|          39028.0|      33|                  63.4|  14283.79|
| Acura|      Legend 3.2/3.5|1998|          27132.0|      27|                 56.06|  11921.16|
| Acura|      Legend 3.2/3.5|1995|          20393.0|      30|                  59.9|   8177.74|
| Acura|      Legend 3.2/3.5|1993|          15156.0|      32|                 62.27|   5718.47|
| Acura|      Legend 3.2/3.5|1992|          14440.0|      33|                  63.4|   5284.87|
| Acura|      Integra GS 1.8|1991|      

In [37]:
df_marca.orderBy(asc('valor_2025')).show(10)

+-----+--------------------+----+-----------------+--------+----------------------+----------+
|marca|              modelo| ano|valor_tabela_fipe|anos_uso|percentual_depreciacao|valor_2025|
+-----+--------------------+----+-----------------+--------+----------------------+----------+
| Lada|           Laika 1.5|1991|           1831.0|      34|                  64.5|    650.02|
| Lada|           Laika 1.5|1991|           1845.0|      34|                  64.5|    654.99|
| Lada|           Laika 1.5|1991|           1884.0|      34|                  64.5|    668.84|
| Lada|           Laika 1.5|1991|           1887.0|      34|                  64.5|     669.9|
| Lada|           Laika 1.5|1991|           1896.0|      34|                  64.5|     673.1|
| Ford|Del Rey L 1.8 / 1...|1985|           2297.0|      40|                 70.43|    679.25|
| Lada|           Laika 1.5|1991|           1921.0|      34|                  64.5|    681.97|
| Ford|Del Rey L 1.8 / 1...|1985|           2315.0

In [38]:
df_marca.orderBy(desc('valor_2025')).show(10, truncate=False)

+-----------+--------------------------------------+----+-----------------+--------+----------------------+----------+
|marca      |modelo                                |ano |valor_tabela_fipe|anos_uso|percentual_depreciacao|valor_2025|
+-----------+--------------------------------------+----+-----------------+--------+----------------------+----------+
|Ferrari    |SF 90 SPIDER 4.0 V8 Bi-Turbo (Híbrido)|2022|8033191.0        |3       |8.73                  |7331676.53|
|Rolls-Royce|Cullinan Black Badge 6.7 V12 Aut.     |2022|6612510.0        |3       |8.73                  |6035059.34|
|Rolls-Royce|Cullinan Black Badge 6.7 V12 Aut.     |2021|6056916.0        |4       |11.47                 |5362144.19|
|Rolls-Royce|Phantom 6.7 V12 Aut.                  |2019|5997983.0        |6       |16.7                  |4996151.93|
|Ferrari    |F8 Spider 3.9 V8 Bi-Turbo 720cv       |2020|4429998.0        |5       |14.13                 |3804190.02|
|Ferrari    |F8 Spider 3.9 V8 Bi-Turbo 720cv    

**Considerando os valores em 2025**

Podemos verificar que o carro mais barato seria o Laika 1.5 da marca Lada, anunciado por cerca de $ 650.

Já o mais caro seria o SF 90 SPIDER 4.0 V8 Bi-Turbo (Híbrido) da Ferrari, custando cerca de $ 7331676.

#### **Verificando os carros mais caros por marca**

In [45]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# separando por marca e ordenando pelo mais caro
janela = Window.partitionBy("marca").orderBy(desc("valor_tabela_fipe"))

# criando a coluna de rank
df_ranked = df.withColumn("rank", row_number().over(janela))

# filtrando apenas o carro mais caro de cada marca
df_mais_caros = df_ranked.filter(col("rank") == 1).orderBy(desc("valor_tabela_fipe"))

In [46]:
df_mais_caros.show(truncate=False)

+--------------+----------------------------------------+-----------+----+-----------------+-----------------+---+-------------------+-----------------------+-------+----+
|marca         |modelo                                  |engine_size|ano |valor_tabela_fipe|flag_veiculo_novo|uf |data_cadastro      |cor                    |placa  |rank|
+--------------+----------------------------------------+-----------+----+-----------------+-----------------+---+-------------------+-----------------------+-------+----+
|Ferrari       |SF 90 SPIDER 4.0 V8 Bi-Turbo (Híbrido)  |4.0        |2022|8033191.0        |False            |PE |2023-02-21 22:18:03|Alizarina              |CX9T328|1   |
|Rolls-Royce   |Cullinan Black Badge 6.7 V12 Aut.       |6.7        |2022|6612510.0        |False            |RO |2023-01-11 15:29:10|Ciano claro            |JF1L924|1   |
|Mclaren       |720S Coupe 4.0 Bi-Turbo V8 2p           |4.0        |2021|4004239.0        |False            |GO |2022-10-23 08:01:11|Urucum

Podemos observar que as 3 marcas que possuem carros mais caros são:

1- Ferrari <br>
2- Rolls-Royce <br>
3- Mclaren <br>

Com carros de mais de 4 milhões de dólares.

#### Filtando somente os carros de mais de 1M

In [48]:
df.filter(col('valor_tabela_fipe') >= 1000000).orderBy(desc('valor_tabela_fipe')).show()

+-----------+--------------------+-----------+----+-----------------+-----------------+---+-------------------+--------------+-------+
|      marca|              modelo|engine_size| ano|valor_tabela_fipe|flag_veiculo_novo| uf|      data_cadastro|           cor|  placa|
+-----------+--------------------+-----------+----+-----------------+-----------------+---+-------------------+--------------+-------+
|    Ferrari|SF 90 SPIDER 4.0 ...|        4.0|2022|        8033191.0|            False| PE|2023-02-21 22:18:03|     Alizarina|CX9T328|
|Rolls-Royce|Cullinan Black Ba...|        6.7|2022|        6612510.0|            False| RO|2023-01-11 15:29:10|   Ciano claro|JF1L924|
|Rolls-Royce|Cullinan Black Ba...|        6.7|2021|        6056916.0|            False| PE|2023-01-20 02:54:02|         Oliva|L0DK1SV|
|Rolls-Royce|Phantom 6.7 V12 Aut.|        6.7|2019|        5997983.0|            False| AM|2021-01-09 10:37:34|    Rosa claro|T3ZAKCO|
|    Ferrari|F8 Spider 3.9 V8 ...|        3.9|2020|    

#### **Criando novos DF manipulando as colunas**

In [50]:
from pyspark.sql.functions import avg,max,min,sum,count, stddev

In [59]:
df_agg_marca = df.groupBy("marca") \
          .agg(min("valor_tabela_fipe").alias("minimo_fipe"),
               avg("valor_tabela_fipe").alias("media_fipe"),
               max("valor_tabela_fipe").alias("maximo_fipe"),
               min("ano").alias("minimo_ano"),
               max("ano").alias("maximo_ano"),
               count("*").alias("qtd_carros_avaliados"))

df_agg_marca.show()

+-----------+-----------+------------------+-----------+----------+----------+--------------------+
|      marca|minimo_fipe|        media_fipe|maximo_fipe|minimo_ano|maximo_ano|qtd_carros_avaliados|
+-----------+-----------+------------------+-----------+----------+----------+--------------------+
|      IVECO|   122296.0|167906.44444444444|   225530.0|      2019|      2022|                  18|
|     AM Gen|   303653.0| 404746.3541666667|   499632.0|      1998|      2000|                  48|
|Asia Motors|     9545.0| 19205.52525252525|    30215.0|      1994|      1999|                  99|
|    Peugeot|     3062.0| 22548.58281998631|   205769.0|      1992|      2022|                2922|
|      Lexus|    10785.0| 90296.56060606061|   896982.0|      1991|      2022|                 330|
|     Jaguar|    35563.0|177671.93073593074|   707815.0|      1994|      2021|                 462|
|        TAC|    76308.0| 94053.63157894737|   112741.0|      2010|      2019|                  19|


In [60]:
# salvando o dataframe gerado
df_agg_marca.write.parquet("/content/df_agg_marca")

In [61]:
# lendo o dataframe para utilizar o spark SQL para pesquisas
df_00 = spark.read.parquet("/content/df_agg_marca")

### **Spark SQL**

In [68]:
# criando uma view temporária em memória para criar consultas em linguagem SQL
df_00.createOrReplaceTempView("df_00")

In [63]:
# verificando o dataframe
spark.sql("""
SELECT *
FROM df_00
""").show()

+-----------+-----------+------------------+-----------+----------+----------+--------------------+
|      marca|minimo_fipe|        media_fipe|maximo_fipe|minimo_ano|maximo_ano|qtd_carros_avaliados|
+-----------+-----------+------------------+-----------+----------+----------+--------------------+
|      IVECO|   122296.0|167906.44444444444|   225530.0|      2019|      2022|                  18|
|     AM Gen|   303653.0| 404746.3541666667|   499632.0|      1998|      2000|                  48|
|Asia Motors|     9545.0| 19205.52525252525|    30215.0|      1994|      1999|                  99|
|    Peugeot|     3062.0| 22548.58281998631|   205769.0|      1992|      2022|                2922|
|      Lexus|    10785.0| 90296.56060606061|   896982.0|      1991|      2022|                 330|
|     Jaguar|    35563.0|177671.93073593074|   707815.0|      1994|      2021|                 462|
|        TAC|    76308.0| 94053.63157894737|   112741.0|      2010|      2019|                  19|


In [66]:
# verificando quais marcas tem a fipe mínima acima de 1M
spark.sql("""
SELECT *
FROM df_00
WHERE minimo_fipe > 1000000
""").show()

+-----------+-----------+------------------+-----------+----------+----------+--------------------+
|      marca|minimo_fipe|        media_fipe|maximo_fipe|minimo_ano|maximo_ano|qtd_carros_avaliados|
+-----------+-----------+------------------+-----------+----------+----------+--------------------+
|Rolls-Royce|  1354027.0|3143156.6666666665|  6612510.0|      2011|      2022|                  33|
|    Mclaren|  1577604.0|2675326.4615384615|  4004239.0|      2017|      2021|                  13|
+-----------+-----------+------------------+-----------+----------+----------+--------------------+



In [67]:
# verificando o valor mínimo da Ferrari
spark.sql("""
SELECT *
FROM df_00
WHERE marca == 'Ferrari'
""").show()

+-------+-----------+-----------------+-----------+----------+----------+--------------------+
|  marca|minimo_fipe|       media_fipe|maximo_fipe|minimo_ano|maximo_ano|qtd_carros_avaliados|
+-------+-----------+-----------------+-----------+----------+----------+--------------------+
|Ferrari|   122439.0|1430347.569105691|  8033191.0|      1993|      2022|                 123|
+-------+-----------+-----------------+-----------+----------+----------+--------------------+



Apesar de ter o carro mais caro, a Ferrari possui o valor mínimo de pouco mais de $ 100.000,00

#### Utilizando Spark SQL para manipular a tabela base e criar novas colunas

In [69]:
df_base = spark.read.parquet("/content/dados_veiculos_sample")

In [70]:
df_base.show()

+------+--------------------+-----------+----+-----------------+-----------------+---+-------------------+--------------------+-------+-----------------+
| marca|              modelo|engine_size| ano|valor_tabela_fipe|flag_veiculo_novo| uf|      data_cadastro|                 cor|  placa|__index_level_0__|
+------+--------------------+-----------+----+-----------------+-----------------+---+-------------------+--------------------+-------+-----------------+
| Acura|             NSX 3.0|        3.0|1995|          43779.0|            False| MT|2005-09-02 14:40:54|      Orquídea média|4RR2KR0|                0|
| Acura|             NSX 3.0|        3.0|1994|          42244.0|            False| MT|1998-11-01 11:58:46|            Azul aço|5C6NIZ1|                1|
| Acura|             NSX 3.0|        3.0|1992|          39028.0|            False| AP|2014-07-03 20:32:21|            Ametista|3PRJCPK|                3|
| Acura|      Legend 3.2/3.5|        3.2|1998|          27132.0|            

In [74]:
# removendo a coluna que não agrega
df_base = df_base.drop('__index_level_0__')
df_base.show()

+------+--------------------+-----------+----+-----------------+-----------------+---+-------------------+--------------------+-------+
| marca|              modelo|engine_size| ano|valor_tabela_fipe|flag_veiculo_novo| uf|      data_cadastro|                 cor|  placa|
+------+--------------------+-----------+----+-----------------+-----------------+---+-------------------+--------------------+-------+
| Acura|             NSX 3.0|        3.0|1995|          43779.0|            False| MT|2005-09-02 14:40:54|      Orquídea média|4RR2KR0|
| Acura|             NSX 3.0|        3.0|1994|          42244.0|            False| MT|1998-11-01 11:58:46|            Azul aço|5C6NIZ1|
| Acura|             NSX 3.0|        3.0|1992|          39028.0|            False| AP|2014-07-03 20:32:21|            Ametista|3PRJCPK|
| Acura|      Legend 3.2/3.5|        3.2|1998|          27132.0|            False| AP|2018-01-07 20:03:26|            Caramelo|2FXXZ1W|
| Acura|      Legend 3.2/3.5|        3.2|1995|  

In [73]:
# criando a view para editar via spark sql
df_base.createOrReplaceTempView("df_base")

Criando os valores mínimos, máximos e média da tabela fipe por marca

In [76]:
spark.sql("""
SELECT marca,
       ROUND(MIN(valor_tabela_fipe), 2) AS minimo_fipe,
       ROUND(MAX(valor_tabela_fipe), 2) AS maximo_fipe,
       ROUND(AVG(valor_tabela_fipe), 2) AS media_fipe
FROM df_base
GROUP BY marca
ORDER BY media_fipe DESC
""").show()

+-------------+-----------+-----------+----------+
|        marca|minimo_fipe|maximo_fipe|media_fipe|
+-------------+-----------+-----------+----------+
|  Rolls-Royce|  1354027.0|  6612510.0|3143156.67|
|      Mclaren|  1577604.0|  4004239.0|2675326.46|
|      Ferrari|   122439.0|  8033191.0|1430347.57|
| ASTON MARTIN|   456992.0|  1952999.0| 792051.68|
|       AM Gen|   303653.0|   499632.0| 404746.35|
|      Porsche|    41580.0|  3571135.0| 391692.62|
|          RAM|   199740.0|   516056.0| 324307.07|
|     Maserati|    19501.0|  1890000.0| 300025.57|
|   CAB Motors|   208627.0|   208627.0|  208627.0|
|       Jaguar|    35563.0|   707815.0| 177671.93|
|        IVECO|   122296.0|   225530.0| 167906.44|
|   Land Rover|    14848.0|  1405166.0| 139020.71|
|Mercedes-Benz|     9484.0|  2009249.0| 134366.17|
|          BMW|    10941.0|  1270339.0| 111682.06|
|       LOBINI|    90531.0|   142521.0| 107202.57|
|         MINI|    41616.0|   277963.0| 101148.01|
|          TAC|    76308.0|   1

Verificando a quantidade de veículos e a soma do preço desses veiculos por estado

In [81]:
spark.sql("""
SELECT uf, COUNT(*) AS qtd_veiculos, SUM(valor_tabela_fipe) AS total_valor_veiculos
FROM df_base
GROUP BY uf
ORDER BY qtd_veiculos DESC
""").show()

+---+------------+--------------------+
| uf|qtd_veiculos|total_valor_veiculos|
+---+------------+--------------------+
| CE|        5559|        2.83449348E8|
| GO|        2905|        1.71870361E8|
| MG|        2902|        1.62613317E8|
| PA|        2897|        1.60733342E8|
| PB|        2878|        1.61607916E8|
| PI|        2852|        1.52668935E8|
| AM|        2850|        1.68644352E8|
| MT|        2844|        1.57133397E8|
| MA|        2830|        1.49565143E8|
| PR|        2820|        1.52884779E8|
| ES|        2810|        1.57075888E8|
| AC|        2800|        1.56205348E8|
| RS|        2797|        1.63858961E8|
| AL|        2792|        1.57805047E8|
| RR|        2788|        1.53997902E8|
| MS|        2781|        1.48390567E8|
| DF|        2779|        1.52593281E8|
| BA|        2765|        1.60997523E8|
| AP|        2756|        1.45238793E8|
| SP|        2744|         1.4697298E8|
+---+------------+--------------------+
only showing top 20 rows



Verificando a quantidade de cadastros por ano

In [83]:
from pyspark.sql.functions import year, month, dayofmonth, hour

In [84]:
spark.sql("""
SELECT YEAR(data_cadastro) AS ano_cadastro, COUNT(*) AS qtd_cadastros
FROM df_base
GROUP BY ano_cadastro
ORDER BY ano_cadastro DESC
""").show()

+------------+-------------+
|ano_cadastro|qtd_cadastros|
+------------+-------------+
|        2023|         1178|
|        2022|         5092|
|        2021|         4855|
|        2020|         4754|
|        2019|         4511|
|        2018|         4183|
|        2017|         4090|
|        2016|         3956|
|        2015|         3560|
|        2014|         3271|
|        2013|         3229|
|        2012|         2992|
|        2011|         2854|
|        2010|         2555|
|        2009|         2386|
|        2008|         2262|
|        2007|         2123|
|        2006|         1836|
|        2005|         1801|
|        2004|         1646|
+------------+-------------+
only showing top 20 rows



Verificando quais carros são novos e quais são mais antigos seguindo os seguintes parâmetros:

ano >= 2015 = Novo

Ano < 2015 = Antigo

In [87]:
spark.sql("""
SELECT
  *,
  CASE
    WHEN YEAR(data_cadastro) >= 2015 THEN 'novo'
    ELSE 'antigo'
  END AS categoria_carro
FROM df_base
""").show()

+------+--------------------+-----------+----+-----------------+-----------------+---+-------------------+--------------------+-------+---------------+
| marca|              modelo|engine_size| ano|valor_tabela_fipe|flag_veiculo_novo| uf|      data_cadastro|                 cor|  placa|categoria_carro|
+------+--------------------+-----------+----+-----------------+-----------------+---+-------------------+--------------------+-------+---------------+
| Acura|             NSX 3.0|        3.0|1995|          43779.0|            False| MT|2005-09-02 14:40:54|      Orquídea média|4RR2KR0|         antigo|
| Acura|             NSX 3.0|        3.0|1994|          42244.0|            False| MT|1998-11-01 11:58:46|            Azul aço|5C6NIZ1|         antigo|
| Acura|             NSX 3.0|        3.0|1992|          39028.0|            False| AP|2014-07-03 20:32:21|            Ametista|3PRJCPK|         antigo|
| Acura|      Legend 3.2/3.5|        3.2|1998|          27132.0|            False| AP|20

Verificando os 10 modelos mais caros

In [90]:
spark.sql("""
SELECT marca, modelo, valor_tabela_fipe
FROM df_base
ORDER BY valor_tabela_fipe DESC
LIMIT 10
""").show(truncate=False)

+-----------+--------------------------------------+-----------------+
|marca      |modelo                                |valor_tabela_fipe|
+-----------+--------------------------------------+-----------------+
|Ferrari    |SF 90 SPIDER 4.0 V8 Bi-Turbo (Híbrido)|8033191.0        |
|Rolls-Royce|Cullinan Black Badge 6.7 V12 Aut.     |6612510.0        |
|Rolls-Royce|Cullinan Black Badge 6.7 V12 Aut.     |6056916.0        |
|Rolls-Royce|Phantom 6.7 V12 Aut.                  |5997983.0        |
|Ferrari    |F8 Spider 3.9 V8 Bi-Turbo 720cv       |4429998.0        |
|Ferrari    |F8 Spider 3.9 V8 Bi-Turbo 720cv       |4423353.0        |
|Ferrari    |488 Pista Spider 3.9 V8 720cv         |4117401.0        |
|Ferrari    |GTC4 LUSSO 6.3 V12 690cv              |4092920.0        |
|Mclaren    |720S Coupe 4.0 Bi-Turbo V8 2p         |4004239.0        |
|Mclaren    |720S Coupe 4.0 Bi-Turbo V8 2p         |3998232.0        |
+-----------+--------------------------------------+-----------------+



Criando faixas de tamanho de motor e verificando a média de preço de cada uma

In [94]:
spark.sql("""
SELECT CASE
  WHEN engine_size < 1.6 THEN 'Pequeno'
  WHEN engine_size < 3.0 THEN 'Medio'
  WHEN engine_size >= 3.0 THEN 'Grande'
END AS tamanho_motor,
AVG(valor_tabela_fipe) AS media_preco
FROM df_base
GROUP BY tamanho_motor
ORDER BY media_preco DESC
""").show(truncate=False)

+-------------+------------------+
|tamanho_motor|media_preco       |
+-------------+------------------+
|Grande       |126053.38773107265|
|Medio        |35378.09391052378 |
|Pequeno      |24824.16883116883 |
+-------------+------------------+



Verificando a quantidade de cores disponíveis em cada marca

In [95]:
spark.sql("""
SELECT marca, COUNT(DISTINCT cor) AS qtd_cores
FROM df_base
GROUP BY marca
ORDER BY qtd_cores DESC
""").show(truncate=False)

+---------------+---------+
|marca          |qtd_cores|
+---------------+---------+
|Peugeot        |227      |
|Mitsubishi     |227      |
|Audi           |227      |
|Mercedes-Benz  |227      |
|Renault        |227      |
|BMW            |227      |
|VW - VolksWagen|227      |
|Citroën        |227      |
|GM - Chevrolet |227      |
|Toyota         |227      |
|Nissan         |227      |
|Fiat           |227      |
|Ford           |227      |
|Honda          |226      |
|Kia Motors     |226      |
|Volvo          |226      |
|Hyundai        |226      |
|Subaru         |226      |
|Land Rover     |225      |
|Porsche        |225      |
+---------------+---------+
only showing top 20 rows



Verificando se existem placas duplicadas neste base

In [96]:
spark.sql("""
SELECT placa, COUNT(*) AS qtd_placas
FROM df_base
GROUP BY placa
HAVING COUNT(*) > 1
""").show(truncate=False)

+-----+----------+
|placa|qtd_placas|
+-----+----------+
+-----+----------+



Aparentemente não temos placas duplicadas na base, mas podemos verificar utilizando outra query

In [97]:
spark.sql("""
SELECT placa, COUNT(*) AS qtd_placas
FROM df_base
GROUP BY placa
ORDER BY qtd_placas DESC
""").show(truncate=False)

+-------+----------+
|placa  |qtd_placas|
+-------+----------+
|L9JW2Y7|1         |
|PSTV5DG|1         |
|VABZFWC|1         |
|NX7RZYJ|1         |
|J8O27PD|1         |
|7O0EK7K|1         |
|CJVUXGB|1         |
|FI2QENN|1         |
|K8S0FMA|1         |
|B3OGV71|1         |
|4B2RQ21|1         |
|WVQ9X77|1         |
|RGIEBMY|1         |
|AK615GF|1         |
|22NARQJ|1         |
|FU6LXPU|1         |
|Q1RB9U9|1         |
|RPGE3RO|1         |
|X86SAKH|1         |
|HZSI4TW|1         |
+-------+----------+
only showing top 20 rows



## **Conclusão**

O Spark SQL é uma ferramenta prática e poderosa para explorar e transformar dados de forma eficiente.

Com ele é possível aplicar filtros, agrupamentos, cálculos e classificações de maneira clara e performática, mesmo em grandes volumes de dados.

É uma solução que combina a familiaridade da linguagem SQL com a escalabilidade do Spark, sendo muito útil no dia a dia de quem trabalha com dados.